In [1]:
%matplotlib qt 

import numpy as np
import pandas as pd
import random
import time
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from imblearn.metrics import specificity_score

from scipy.stats import iqr
from scipy.stats import t

In [2]:
# import data
data = pd.read_csv('bm_combat_spectral_changed.csv', index_col=[0])

# non-harmonized 
data_log = data.drop(data.iloc[:, 6:209], axis=1)

In [9]:
np.random.seed(0)
random.seed(0)

X = data_log
y = data_log[['center','label']]

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.7,
                                                    random_state=1,
                                                    stratify=X[['center','group']])

In [12]:
center_names = ['all','california','finland','iowa','medellin']

X_train_sets = {} 
y_train_sets = {}
X_test_sets = {}
y_test_sets = {}


for name in center_names:
    
    if name == 'all':
        X_train_sets.update({name: X_train.drop(['center', 'group', 'age', 'gender', 'batch', 'label'], axis=1)})
        X_test_sets.update({name: X_test.drop(['center', 'group', 'age', 'gender', 'batch', 'label'], axis=1)})
        y_train_sets.update({name: y_train.drop(['center'], axis = 1)})
        y_test_sets.update({name: y_test.drop(['center'], axis = 1)})
    
    else: 
        X_train_set = X_train.loc[X_train['center'] == name]
        X_train_set = X_train_set.drop(['center', 'group', 'age', 'gender', 'batch', 'label'], axis=1)
        X_train_sets.update({name: X_train_set})

        y_train_set = y_train.loc[X_train['center'] == name]
        y_train_set = y_train_set.drop(['center'], axis=1)
        y_train_sets.update({name: y_train_set})

        X_test_set = X_test.loc[X_test['center'] == name]
        X_test_set = X_test_set.drop(['center', 'group', 'age', 'gender', 'batch', 'label'], axis=1)
        X_test_sets.update({name: X_test_set})

        y_test_set = y_test.loc[X_test['center'] == name]
        y_test_set = y_test_set.drop(['center'], axis=1)
        y_test_sets.update({name: y_test_set})

In [17]:
# Initializing Classifiers

clf1 = LogisticRegression(multi_class='multinomial',
                          solver='newton-cg',
                          random_state=1)

clf2 = KNeighborsClassifier(algorithm='ball_tree',
                            leaf_size=50)

clf3 = DecisionTreeClassifier(random_state=1)

clf4 = SVC(random_state=1)

# Building the pipelines

pipe1 = Pipeline([('clf1', clf1)])

pipe2 = Pipeline([('clf2', clf2)])

pipe4 = Pipeline([('clf4', clf4)])


# Setting up the parameter grids
param_grid1 = [{'clf1__penalty': ['l2'],
                'clf1__C': np.power(10., np.arange(-4, 4))}]

param_grid2 = [{'clf2__n_neighbors': list(range(1, 10)),
                'clf2__p': [1, 2]}]

param_grid3 = [{'max_depth': list(range(1, 10)) + [None],
                'criterion': ['gini', 'entropy']}]

param_grid4 = [
               {'clf4__kernel': ['rbf'],
                'clf4__C': np.power(10., np.arange(-4, 4)),
                'clf4__gamma': np.power(10., np.arange(-5, 0))},
               {'clf4__kernel': ['linear'],
                'clf4__C': np.power(10., np.arange(-4, 4))}
              ]

In [18]:
# Setting up multiple GridSearchCV objects, 1 for each algorithm

gridcvs = {}

for pgrid, est, name in zip((param_grid1, param_grid2,
                             param_grid3, param_grid4),
                            (pipe1, pipe2, clf3, pipe4),
                            ('Softmax', 'KNN', 'DTree', 'SVM')):
    
    gcv = GridSearchCV(estimator=est,
                       param_grid=pgrid,
                       scoring='accuracy',
                       n_jobs=1,
                       cv=5,
                       verbose=0,
                       refit=True)
    gridcvs[name] = gcv

# WITHOUT FEATURE SELECTION

In [20]:
metrics = ['accuracy','recall','specificity','precision','F1','auc']
cv_scores = {center_name: {name: {metric: [] for metric in metrics} for name, gs_est in gridcvs.items()} for center_name in center_names}
cv_scores_without_zeros = {center_name: {name: {metric: [] for metric in metrics} for name, gs_est in gridcvs.items()} for center_name in center_names}

skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

for center_name in center_names:
    
    if center_name != center_names[0]: continue
    
    print('Cross-validation for ' + center_name.upper())

    # The outer loop for algorithm selection
    c = 1
    for outer_train_idx, outer_valid_idx in skfold.split(X_train_sets[center_name],y_train_sets[center_name]):

        for name, gs_est in sorted(gridcvs.items()):
            print('outer fold %d/5 | tuning %-8s' % (c, name), end='')

            # The inner loop for hyperparameter tuning
            gs_est.fit(X_train_sets[center_name].iloc[outer_train_idx.tolist()], y_train_sets[center_name].iloc[outer_train_idx.tolist()].values.ravel())
            y_pred = gs_est.predict(X_train_sets[center_name].iloc[outer_valid_idx.tolist()])
            
            for metric in metrics: 
                if metric == 'accuracy':
                    calc_metric = accuracy_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_pred=y_pred)
                    print(' | inner ACC %.2f%% | outer ACC %.2f%%' %
                          (gs_est.best_score_ * 100, calc_metric * 100))
                    cv_scores[center_name][name][metric].append(calc_metric)
                    if calc_metric != 0:
                        cv_scores_without_zeros[center_name][name][metric].append(calc_metric)
                elif metric == 'recall':
                    calc_metric = recall_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_pred=y_pred)
                    cv_scores[center_name][name][metric].append(calc_metric)
                    if calc_metric != 0:
                        cv_scores_without_zeros[center_name][name][metric].append(calc_metric)
                elif metric == 'specificity':
                    calc_metric = specificity_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_pred=y_pred)
                    cv_scores[center_name][name][metric].append(calc_metric)
                    if calc_metric != 0:
                        cv_scores_without_zeros[center_name][name][metric].append(calc_metric)
                elif metric == 'precision':
                    calc_metric = precision_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_pred=y_pred)
                    cv_scores[center_name][name][metric].append(calc_metric)
                    if calc_metric != 0:
                        cv_scores_without_zeros[center_name][name][metric].append(calc_metric)
                elif metric == 'F1':
                    calc_metric = f1_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_pred=y_pred)
                    cv_scores[center_name][name][metric].append(calc_metric)
                    if calc_metric != 0:
                        cv_scores_without_zeros[center_name][name][metric].append(calc_metric)
                else:
                    calc_metric = roc_auc_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_score=y_pred, average = 'macro')
                    cv_scores[center_name][name][metric].append(calc_metric)
                    if calc_metric != 0:
                        cv_scores_without_zeros[center_name][name][metric].append(calc_metric)

        c += 1

Cross-validation for ALL
outer fold 1/5 | tuning DTree    | inner ACC 70.41% | outer ACC 62.50%
outer fold 1/5 | tuning KNN      | inner ACC 67.19% | outer ACC 62.50%
outer fold 1/5 | tuning SVM      | inner ACC 74.44% | outer ACC 66.67%
outer fold 1/5 | tuning Softmax  | inner ACC 72.28% | outer ACC 79.17%
outer fold 2/5 | tuning DTree    | inner ACC 77.49% | outer ACC 45.83%
outer fold 2/5 | tuning KNN      | inner ACC 74.33% | outer ACC 50.00%
outer fold 2/5 | tuning SVM      | inner ACC 76.49% | outer ACC 54.17%
outer fold 2/5 | tuning Softmax  | inner ACC 78.60% | outer ACC 54.17%
outer fold 3/5 | tuning DTree    | inner ACC 65.96% | outer ACC 75.00%
outer fold 3/5 | tuning KNN      | inner ACC 70.18% | outer ACC 66.67%
outer fold 3/5 | tuning SVM      | inner ACC 76.61% | outer ACC 62.50%
outer fold 3/5 | tuning Softmax  | inner ACC 75.56% | outer ACC 58.33%
outer fold 4/5 | tuning DTree    | inner ACC 66.32% | outer ACC 73.91%
outer fold 4/5 | tuning KNN      | inner ACC 65.26% 

In [21]:
# Looking at the results

mean_std_cv_scores = {center_name: {name: {metric: {'mean': None, 'SD': None} for metric in metrics} for name, gs_est in gridcvs.items()} for center_name in center_names}
mean_std_cv_scores_without_zeros = {center_name: {name: {metric: {'mean': None, 'SD': None} for metric in metrics} for name, gs_est in gridcvs.items()} for center_name in center_names}
median_iqr_cv_scores = {center_name: {name: {metric: {'median': None, 'IQR': None} for metric in metrics} for name, gs_est in gridcvs.items()} for center_name in center_names}
median_iqr_cv_scores_without_zeros = {center_name: {name: {metric: {'median': None, 'IQR': None} for metric in metrics} for name, gs_est in gridcvs.items()} for center_name in center_names}

for center_name in center_names:
    for name, gs_est in sorted(gridcvs.items()):
        print('results for ' + center_name.upper() + ' ' + name.upper())
        for metric in metrics: 
            if metric != 'auc':
                print('%-8s | outer CV ' + metric + ' %.2f%% +\- %.3f' % (
                  100 * np.mean(cv_scores[center_name][name][metric]), 100 * np.std(cv_scores[center_name][name][metric])))
                mean_std_cv_scores[center_name][name][metric]['mean'] = np.mean(cv_scores[center_name][name][metric])
                mean_std_cv_scores_without_zeros[center_name][name][metric]['mean'] = np.mean(cv_scores_without_zeros[center_name][name][metric])
                mean_std_cv_scores[center_name][name][metric]['SD'] = np.std(cv_scores[center_name][name][metric])
                mean_std_cv_scores_without_zeros[center_name][name][metric]['SD'] = np.std(cv_scores_without_zeros[center_name][name][metric])
                
                median_iqr_cv_scores[center_name][name][metric]['median'] = 100*np.median(cv_scores[center_name][name][metric])
                median_iqr_cv_scores_without_zeros[center_name][name][metric]['median'] = 100*np.median(cv_scores_without_zeros[center_name][name][metric])
                median_iqr_cv_scores[center_name][name][metric]['IQR'] = 100*iqr(cv_scores[center_name][name][metric])
                median_iqr_cv_scores[center_name][name][metric]['IQR'] = 100*iqr(cv_scores_without_zeros[center_name][name][metric])
            else: 
                print('%-8s | outer CV ' + metric + ' %.2f +\- %.3f' % (
                  np.mean(cv_scores[center_name][name][metric]), np.std(cv_scores[center_name][name][metric])))
                mean_std_cv_scores[center_name][name][metric]['mean'] = np.mean(cv_scores[center_name][name][metric])
                mean_std_cv_scores_without_zeros[center_name][name][metric]['mean'] = np.mean(cv_scores_without_zeros[center_name][name][metric])
                mean_std_cv_scores[center_name][name][metric]['SD'] = np.std(cv_scores[center_name][name][metric])
                mean_std_cv_scores_without_zeros[center_name][name][metric]['SD'] = np.std(cv_scores_without_zeros[center_name][name][metric])
                
                median_iqr_cv_scores[center_name][name][metric]['median'] = 100*np.median(cv_scores[center_name][name][metric])
                median_iqr_cv_scores_without_zeros[center_name][name][metric]['median'] = 100*np.median(cv_scores_without_zeros[center_name][name][metric])
                median_iqr_cv_scores[center_name][name][metric]['IQR'] = 100*iqr(cv_scores[center_name][name][metric])
                median_iqr_cv_scores[center_name][name][metric]['IQR'] = 100*iqr(cv_scores_without_zeros[center_name][name][metric])
    print('*********************')
print('\nSoftmax Best parameters', gridcvs['Softmax'].best_params_)
print('\nKNN Best parameters', gridcvs['KNN'].best_params_)
print('\nDTree Best parameters', gridcvs['DTree'].best_params_)
print('\nSVM Best parameters', gridcvs['SVM'].best_params_)

results for ALL DTREE
%-8s | outer CV accuracy 62.75% +\- 10.952
%-8s | outer CV recall 49.09% +\- 23.045
%-8s | outer CV specificity 76.52% +\- 20.575
%-8s | outer CV precision 71.34% +\- 19.379
%-8s | outer CV F1 54.65% +\- 16.851
%-8s | outer CV auc 0.63 +\- 0.106
results for ALL KNN
%-8s | outer CV accuracy 61.92% +\- 6.701
%-8s | outer CV recall 59.39% +\- 13.083
%-8s | outer CV specificity 64.55% +\- 11.905
%-8s | outer CV precision 63.17% +\- 7.000
%-8s | outer CV F1 60.48% +\- 7.619
%-8s | outer CV auc 0.62 +\- 0.066
results for ALL SVM
%-8s | outer CV accuracy 65.36% +\- 11.659
%-8s | outer CV recall 58.18% +\- 21.499
%-8s | outer CV specificity 72.88% +\- 12.219
%-8s | outer CV precision 67.60% +\- 9.954
%-8s | outer CV F1 61.15% +\- 15.393
%-8s | outer CV auc 0.66 +\- 0.116
results for ALL SOFTMAX
%-8s | outer CV accuracy 69.64% +\- 11.353
%-8s | outer CV recall 64.70% +\- 17.491
%-8s | outer CV specificity 74.85% +\- 17.338
%-8s | outer CV precision 72.91% +\- 13.400
%-8s |

/Users/2923911/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/2923911/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/2923911/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/2923911/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/Users/2923911/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [22]:
confedence_interval = {center_name: {name: {metric: () for metric in metrics} for name, gs_est in gridcvs.items()} for center_name in center_names}

dof = 5-1 
confidence = 0.95

for center_name in center_names:
    for name, gs_est in sorted(gridcvs.items()):
        for metric in metrics:
    

            m = mean_std_cv_scores[center_name][name][metric]['mean']
            s = mean_std_cv_scores[center_name][name][metric]['SD']

            t_crit = np.abs(t.ppf((1-confidence)/2,dof))
            confedence_interval[center_name][name][metric]= ((m-s*t_crit/np.sqrt(5))*100, (m+s*t_crit/np.sqrt(5))*100)

{'all': {'Softmax': {'accuracy': (55.540905419121444, 83.73445689971915),
   'recall': (42.97945830743256, 86.41448108650684),
   'specificity': (53.32084982879943, 96.37611986817028),
   'precision': (56.26812281340777, 89.54356549828054),
   'F1': (49.892371499288366, 84.69564003151494),
   'auc': (55.614704520810456, 83.9307500246441)},
  'KNN': {'accuracy': (53.600095795921554, 70.24048391422338),
   'recall': (43.14918673691988, 75.63869205095892),
   'specificity': (49.76310504754877, 79.32780404336033),
   'precision': (54.47501867924029, 71.85831465409302),
   'F1': (51.015893347018924, 69.93648760536205),
   'auc': (53.73373058670848, 70.20566335268546)},
  'DTree': {'accuracy': (49.1552388119702, 76.3520075648414),
   'recall': (20.476451423069168, 77.70536675874902),
   'specificity': (50.96811011210911, 102.06219291819392),
   'precision': (47.273968151660995, 95.39830075590203),
   'F1': (33.7313681204929, 75.57732681433633),
   'auc': (49.676264389231, 75.9297962168296)},

In [ ]:
# BOOTSTRAPPING

from sklearn.utils import resample

# Fitting a model to the whole training set
# using the "best" algorithm
best_algo = gridcvs['Softmax']


bootstrap = {center_name: {metric: [] for metric in metrics} for center_name in center_names}

for center_name in center_names:
    
    print('Test results for ' + center_name.upper())

    n_iterations = 100  # No. of bootstrap samples to be repeated (created)
    n_size = int(len(X_test_sets[center_name]) * 1.0) # Size of sample, picking only 50% of the given data in every bootstrap sample
    
    for i in range(n_iterations):
    
        test_values = resample(X_test_sets[center_name], replace=False, n_samples = n_size)
        test_labels = y_test_sets[center_name].loc[test_values.index.to_list()]

        if center_name == 'all':
            start = time.time()
            best_algo.fit(X_train_sets[center_name], y_train_sets[center_name].values.ravel())
            stop = time.time()

            for metric in metrics: 

                if metric == 'accuracy':
                    calc_metric = accuracy_score(y_true=test_labels, y_pred=best_algo.predict(test_values))
                    print('Test Accuracy: %.2f%%' % (100 * calc_metric))
                    bootstrap[center_name][metric].append(calc_metric)
                elif metric == 'recall':
                    calc_metric = recall_score(y_true=test_labels, y_pred=best_algo.predict(test_values))
                    print('Recall/sensitivity: %.2f%%' % (100 * calc_metric))
                    bootstrap[center_name][metric].append(calc_metric)
                elif metric == 'specificity':
                    calc_metric = specificity_score(y_true=test_labels, y_pred=best_algo.predict(test_values))
                    print('Specificity: %.2f%%' % (100 * calc_metric))
                    bootstrap[center_name][metric].append(calc_metric)
                elif metric == 'precision':
                    calc_metric = precision_score(y_true=test_labels, y_pred=best_algo.predict(test_values))
                    print('Precision: %.2f%%' % (100 * calc_metric))
                    bootstrap[center_name][metric].append(calc_metric)
                elif metric == 'F1':
                    calc_metric = f1_score(y_true=test_labels, y_pred=best_algo.predict(test_values))
                    print('F1 score: %.2f%%' % (100 * calc_metric))
                    bootstrap[center_name][metric].append(calc_metric)
                else:
                    calc_metric = roc_auc_score(y_true=test_labels, y_score=best_algo.predict(test_values), average = 'macro')
                    print('ROC AUC: %.2f' % calc_metric)
                    bootstrap[center_name][metric].append(calc_metric)

        else:

            for metric in metrics: 
                if metric == 'accuracy':
                    calc_metric = accuracy_score(y_true=test_labels, y_pred=best_algo.predict(test_values))
                    print('Test Accuracy: %.2f%%' % (100 * calc_metric))
                    bootstrap[center_name][metric].append(calc_metric)
                elif metric == 'recall':
                    calc_metric = recall_score(y_true=test_labels, y_pred=best_algo.predict(test_values))
                    print('Recall/sensitivity: %.2f%%' % (100 * calc_metric))
                    bootstrap[center_name][metric].append(calc_metric)
                elif metric == 'specificity':
                    calc_metric = specificity_score(y_true=test_labels, y_pred=best_algo.predict(test_values))
                    print('Specificity: %.2f%%' % (100 * calc_metric))
                    bootstrap[center_name][metric].append(calc_metric)
                elif metric == 'precision':
                    calc_metric = precision_score(y_true=test_labels, y_pred=best_algo.predict(test_values))
                    print('Precision: %.2f%%' % (100 * calc_metric))
                    bootstrap[center_name][metric].append(calc_metric)
                elif metric == 'F1':
                    calc_metric = f1_score(y_true=test_labels, y_pred=best_algo.predict(test_values))
                    print('F1 score: %.2f%%' % (100 * calc_metric))
                    bootstrap[center_name][metric].append(calc_metric)
                else:
                    calc_metric = roc_auc_score(y_true=test_labels, y_score=best_algo.predict(test_values), average = 'macro')
                    print('ROC AUC: %.2f' % calc_metric)
                    bootstrap[center_name][metric].append(calc_metric)

In [ ]:
for center_name in center_names:
    print('Results for ' + center_name.upper())
    for metric in metrics: 
        if metric != 'auc':
            print(metric + ' %.2f%% +\- %.3f' % (
              100 * np.mean(bootstrap[center_name][metric]), 100 * np.std(bootstrap[center_name][metric])))
        else: 
            print(metric + ' %.2f +\- %.3f' % (
              np.mean(bootstrap[center_name][metric]), np.std(bootstrap[center_name][metric])))